In [1]:
import xml.etree.ElementTree as ET
import re
from tkinter import filedialog, messagebox,simpledialog
from tkinter import *
from pandastable import Table,TableModel
import pandas as pd
import sys
import matplotlib.pyplot as plt
from tksheet import Sheet
import time
from tabulate import tabulate

class PrintToT1(object): 
    def write(self, s): 
        t1.insert(END, s) 
        t1.tag_add("center", "1.0", "end")

        
def close():
    root.destroy()
    
    
root = Tk() 
root.geometry('600x400')
root.configure(bg='black')
scrollbar = Scrollbar(root)
t1 = Text(root, yscrollcommand=scrollbar.set,bg='black',fg='white')
scrollbar.config(command=t1.yview)
scrollbar.pack(side=RIGHT, fill=Y)    
t1.tag_configure("center", justify='center')
btn2 = Button(root, text ='PRÓX NF',command=close).pack(side=TOP, fill = 'x')  
btn = Button(root, text ='SAIR',command=close).pack(side=TOP, fill = 'y')
t1.pack() 
sys.stdout = PrintToT1()        
    
    
def biel(stri,xi = 'X'):
    ''' retorna o index do ultimo X no nome '''
    xi = xi.upper()
    try:
        qtd = stri[::-1].upper().index(xi)
    except ValueError:
        return False
    qt = len(stri) - qtd
    return qt


def maluco(icms, desc, qtrib, vprod, fcp, ipi, x, nome, dire=''):
    ''' função maluco pega todos os valores do produto do xml e retorna o preço final
    escolhendo a qtd de caixa com base na escolha do usuário e no nome do produto
    val1 vira valo1 qdo só tem números, ele pega o lado esquerdo, do ultimo X encontrado
    val2 vira valo2 qdo só tem números, ele pega o lado direito,  do ultimo X encontrado '''
    valo1, valo2 = '', ''
    val1 = nome[x - 5:x]
    val2 = nome[x:x + 5]
    for i in val1:
        if i.isdigit():
            valo1 += i
    for i in val2:
        if i.isdigit():
            valo2 += i
    if dire == 'e':
        if not valo1:
            return 'un'
        pf = (vprod + icms + fcp + ipi - desc) / (int(valo1) * qtrib)
        return pf, valo1
    if dire == 'd':
        if not valo2:
            return 'un'
        pf = (vprod + icms + fcp + ipi - desc) / (int(valo2) * qtrib)
        return pf, valo2
    if dire == 'nda':
        if not valo2:
            return 'un'
        pf = (vprod + icms + fcp + ipi - desc) / (int(valo2) * qtrib)
        return pf, valo2
    if dire[0] == '0':
        if dire[1:4].lower() == 'esq':
            pf = (vprod + icms + fcp + ipi - desc) / (int(valo1) * qtrib)
            return pf,valo1
        if dire[1:4].lower() == 'dir':
            pf = (vprod + icms + fcp + ipi - desc) / (int(valo2) * qtrib)
            return pf,valo2


def formato(nome, pf, ean, qcx=1):
    global itens
    pv = '{:.2f}'.format(float(pf))
    for i in itens:
        if i['nome'] == nome:
            i['preço'] = pv
            i['ean'] = ean
            i['qcx'] = qcx


def digitos(cx):
    new, qcx = '', 0
    for i in cx:
        if i.isdigit():
            new += i
    try:
        qcx = int(new)
    except ValueError:
        pass
    return qcx


def escolha(ucom, utrib, ean):
    ut = utrib[0:2].lower()
    uti = utrib[0:3].lower()
    qcx = checkEan(ean)
    if ucom and utrib and ucom.lower() == utrib.lower() and utrib.isalpha() == False:
        return 'cx24utrib'
    elif ucom and utrib and ucom.lower() == utrib.lower() and ucom.isalpha() == False:
        return 'cx24ucom'
    elif  (ut == 'un' or uti == 'und' or uti == 'uni' or ut == 'pc' or ut == 'fr' or ut == 'kg' or ut == 'qt' or ut == 'gr'
         or uti == 'lat' or ut == 'lt' or ut == 'pa' or ut == 'cp' or ut == 'pt' or uti == 'pec'):
        return 'un'
    elif qcx and int(qcx) >= 2:
        qcx = int(digitos(qcx))
        return "ean", qcx
    elif (ucom.lower() != ut) and (ut.strip() == 'un' or ut.strip() == 'pc'):
        return 'utrib*ucom'
    else:
        return 'maluco'


def names(rooti):
    global qcx, px, pv, itens,resp,nnf,xfant
    vprod, icms, desc, qtrib, nome, pf, x, utrib, bo, new = 0, 0, 0, 0, '', 0, 0, '', False, ''
    itens, pv, ipi, fcp, ean, qcx, ucom, boc, dire, inf,jaSabe = [], 0, 0, 0, 0, 0, '', 0, '', '',False
    for child in rooti.iter():
        bo = False
        try:
            if child.tag[36:] == 'nNF':
                nnf = child.text
            if child.tag[36:] == 'xFant':
                xfant = child.text
                print('=-=' * 24)
                print(xfant)
                print(nnf)
                if child.text[:9].lower() == 'nova camp':
                    jaSabe = True
                    resp = 'maluco'
                    dire = 'esq'
                if child.text == 'MARSIL':
                    jaSabe = True
                    resp = 'marsil'
            if child.tag[36:] == 'xProd':
                ucom,utrib = '',''
                desc,icms,vprod,pf,ipi,fcp,qcx = 0,0,0,0,0,0,0
                nome = child.text
                itens.append({'nome': nome})
            if child.tag[36:] == 'vProd':
                vprod = float(child.text)
            if child.tag[36:] == 'cEANTrib':
                ean = (child.text)
            if child.tag[36:] == 'uCom':
                ucom = child.text
            if child.tag[36:] == 'uTrib':
                utrib = child.text
            if child.tag[36:] == 'qTrib':
                qtrib = float(child.text)
            if child.tag[36:] == 'vDesc':
                desc = float(child.text)
            if child.tag[36:] == 'vICMSST':
                icms = float(child.text)
            if child.tag[36:] == 'vFCPST':
                fcp = float(child.text)
            if child.tag[36:] == 'vIPI':
                ipi = float(child.text)
            if child.tag[36:] == 'infAdProd':
                inf = child.text
            if child.tag[36:39] == 'det' or child.tag[36:] == 'total':
                bo = True if boc else False
                boc = True
            if bo:
                if not jaSabe:
                    resp = escolha(ucom, utrib, ean)
                    if checkEan(ean) and int(checkEan(ean)) > 1:
                        qcx = checkEan(ean)
                if resp == 'marsil':
                    qcx = int(nome[-6:-2])
                    pf = (vprod + icms + fcp- + ipi - desc) / (qcx * qtrib)
                    formato(nome, pf, ean, qcx)
                    qcx = 0
                if resp[0] == 'ean':
                    pf = (vprod + icms + fcp + ipi - desc) / (resp[1] * qtrib)
                    formato(nome, pf, ean, qcx)
                    qcx = 0
                if qcx and ucom != utrib and utrib.lower().strip() == "un":
                    pf = (vprod + icms + fcp + ipi - desc) / (int(qcx) * qtrib)
                    formato(nome, pf, ean, qcx)
                    qcx = 0
                if resp == 'else':
                    pf = (vprod + icms + fcp + ipi - desc) / qtrib
                    formato(nome, pf, ean)
                if resp == 'maluco':
                    if not jaSabe:
                        dire = simpledialog.askstring('decisão do usuário',f'de qual lado está a qtd? [esq/dir/nda] {nome}\nSe não for X digite 0 (zero) a direção e a letra ex: 0ndaP  ',parent=root)
                    if dire == 'nda':
                        y = biel(inf)
                        if not y:
                            resp = 'un'
                        else:
                            qcx = maluco(icms, desc, qtrib, vprod,
                                         fcp, ipi, dire='nda', x=int(y), nome=inf)
                            pf = qcx[0]
                            qcx = qcx[1]
                            formato(nome, pf, ean, qcx)
                    elif dire == 'dir':
                        x = biel(nome)
                        p = maluco(icms, desc, qtrib, vprod, fcp, ipi, x, nome, dire="d")
                        pf = p[0]
                        qcx = p[1]
                        formato(nome, pf, ean, qcx)
                    elif dire == 'esq':
                        x = biel(nome)
                        p = maluco(icms, desc, qtrib, vprod, fcp, ipi, x, nome, dire="e")
                        pf = p[0]
                        qcx = p[1]
                        formato(nome, pf, ean, qcx)
                    elif dire[0] == '0':
                        x = biel(nome,xi=dire[-1])
                        p = maluco(icms, desc, qtrib, vprod, fcp, ipi, x, nome, dire)
                        pf = p[0]
                        qcx = p[1]
                        formato(nome, pf, ean, qcx)
                    else:
                        break
                if resp == 'cx24utrib':
                    qcx = digitos(utrib)
                    pf = (vprod + icms + fcp + ipi - desc) / (qcx * qtrib)
                    formato(nome, pf, ean, qcx)
                    qcx = 0
                if resp == 'cx24ucom':
                    qcx = digitos(ucom)
                    pf = (vprod + icms + fcp + ipi - desc) / (qcx * qtrib)
                    formato(nome, pf, ean, qcx)
                    qcx = 0
                if resp == 'un':
                    pf = (vprod + icms - desc + fcp + ipi) / qtrib
                    formato(nome, pf, ean)
                if resp == 'utrib*ucom':
                    pf = (vprod + icms - desc + fcp + ipi) / (qtrib)  # <<----- era qcom
                    formato(nome, pf, ean)
            if child.tag[36:] == 'total':
                return itens
        except ZeroDivisionError or IndexError or KeyError:
            continue


def organiza(lista, ordenador, *retorno):
    for i in range(len(lista)):
        lista[i]['nome'] = lista[i]['nome'].replace(",", ";").replace(":", ";").replace("  ", " ")
        lista[i]['nome'] = lista[i]['nome'].replace("  ", " ").replace("  ", " ").replace("  ", " ").replace("  ", " ").replace("  ", " ").replace("  ", " ")
    aux = []
    for dct in lista:
        l = []
        for i in retorno:
            l.append(str(dct[i]))
        plv = "{} : {}".format(dct[ordenador], ','.join(l))
        aux.append(plv)
    aux.sort(key=lambda aux: list(map(float, re.findall(r'\d+', aux)))[0])
    resultado = []
    for i in aux:
        resultado.append(i.split(':')[1].strip().split(','))
    return resultado


def preco(itens):
    '''formata o texto:
    nome do item ========= R$ 25.00'''
    di,lisnome,lispre = {},[],[]
    deu = organiza(itens, 'preço', 'nome', 'preço', 'ean', 'qcx')
    for i in deu:
        n = i[0]
        l = 68 - len(str(n))
        p = (i[1])
        lisnome.append(n)
        lispre.append(f'R${p}')
    di['DESCRIÇÃO DO ITEM'] = lisnome
    di['PREÇO'] = lispre
    return di
        
        
def init():
    '''função de inicialização'''
    xml = filedialog.askopenfiles(title="choosen",filetypes=(("All Files","*"),("Python Script Files","*.py")))
    newteste, itens= [], []
    desc, icms, vprod, qtrib, pf, user, nnf,xfant, string = 0, 0, 0, 0, 0, 0, 0, '',''        
    for i in xml:
        tree = ET.parse(i)
        rooti = tree.getroot()
        di = preco(names(rooti))
        pf = pd.DataFrame(di,columns=['DESCRIÇÃO DO ITEM', 'PREÇO'])
        print(tabulate(pf, showindex=True, headers=pf.columns, tablefmt = 'fancy_grid'))
        root.after(50000,salvare())
 


def salvare():
    r = messagebox.askyesno('bateu', 'os preços bateram?')
    if r :
        salvar()

        
def checkEan(ean):
    ean = str(ean)
    if len(ean) > 13:
        ean = ean[1:]
    with open("txte.txt", 'r') as db:
        for linha in db:
            if linha[:13] == ean[:13]:
                return linha[15:]


def salvar():
    with open("log.txt", 'a') as log:
        for k in itens:
            if int(k['qcx']) >=2:
                log.write(f"\n{k}")

                
x = init()
while x == 1:
    x = init()
root.mainloop() 


numero do xmlsar
Os Testamentos
44.9 <- pf = ( vprod: 44.9 + icms: 0 - desc 0 + fcp 0 + ipi 0 ) / qtrib 1.0
===============================LIVRARIA SARAIVA===============================
=================================NFE: 314345==================================
=Os Testamentos======================================================R$ 44.90=

bateu? [S/N]s
sair? [S/N]s
